In [1]:
import os
import pandas as pd 
import json 

from src.textual_features import exact_keyword_search

# Exact keyword search for paragraphs from BauNVO & BauGB

## Prepare data

- Change the folder path in the code block below to read in the data.
- Specify the relevant column names. The function that is used in the following expects the input data frame to have (at least) two columns, i.e., one id and one content column. Here, the columns are called filename and content. If named differently, change the column names in the code below.


In [2]:
# specify file path
INPUT_FILE_PATH = os.path.join("data", "proc", "building_plans_sample", "bp_text.json")
OUTPUT_FILE_PATH = os.path.join("data", "nrw", "bplan", "features", "keywords", "exact_search", "exact_search.csv")

# specify relevant column names
ID_COLUMN='filename'
TEXT_COLUMN='content'

# read in data
input_df = pd.read_json(INPUT_FILE_PATH)


In [3]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  503 non-null    object
 1   content   503 non-null    object
dtypes: object(2)
memory usage: 8.0+ KB


In [4]:
input_df

,filename,content
0,10197_0.jpg,"[MARKT BÜRGSTADT, Landkreis, Miltenberg, Bebau..."
1,10197_1.jpg,"[FESTSETZUNGEN, Planungsrechtliche Festsetzung..."
2,10197_2.jpg,"[HINWEISE, Brandschutz :, Der 2. Rettungsweg {..."
3,17028_0.jpg,"[b,TEXTTEIL, IIL, VERFAHRENSVERMERKE, L, FESTS..."
4,24451_0.jpg,"[Stadt Ochsenfurt, Stadtteil Goßmannsdorf, Beb..."
...,...,...
498,91911_7.jpg,"[nirna, Schönbe, Brand, häusgr, Moldev, Neumüh..."
499,91911_8.jpg,"[Hoaiul, Gadruckivon han, aLf WTS-46-05pZ14 EM..."
500,91911_9.jpg,"[525, 560, 523, 700, 564, 521, 563, 1, 565, 52..."
501,9581_0.jpg,"[Der Marktgemeinderat hat in der ', sizun?v, v..."


## Define keyword dictionary

Keywords are specified in a separate json file to apply the exact keyword search more easily to different sets of keywords, simply by reading in the relevant dictionary. The dictionary is structured so that each keyword category (e.g. baunvo-1) can contain one or more keywords to consider the category covered (e.g., "§1 baunvo", "1 baunvo", or "allgemeine vorschriften für bauflächen und baugebiete").


In [5]:
with open('dictionaries/keyword_dict_baunvo.json') as f:
    BAUNVO_KEYWORDS = json.load(f)

## Apply function

Exact keyword matching based on input dictionary, returns df showing which keyword appeared in each pdf per category.

In [6]:
result_df = exact_keyword_search.search_df_for_keywords(input_df=input_df,
                                   text_column_name=TEXT_COLUMN,
                                   id_column_name=ID_COLUMN,
                                   keyword_dict=BAUNVO_KEYWORDS)

We can explore the results by seeing the output dataframe:

In [7]:
result_df.head()

,filename,baunvo-1,baunvo-2,baunvo-3,baunvo-4,baunvo-4a,baunvo-5,baunvo-5a,baunvo-6,baunvo-6a,...,baunvo-14,baunvo-15,baunvo-16,baunvo-17,baunvo-18,baunvo-19,baunvo-20,baunvo-21,baunvo-21a,13b
0,10197_0.jpg,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,10197_1.jpg,[1 baunvo],None,None,None,None,None,None,[6 baunvo],None,...,None,[15 baunvo],None,None,[18 baunvo],"[19 baunvo, grz]",[20 baunvo],"[21 baunvo, baumassenzahl, baumasse]",[stellplätze],None
2,10197_2.jpg,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,"[baumassenzahl, baumasse]",None,None
3,17028_0.jpg,[1 baunvo],[2 baunvo],[3 baunvo],[4 baunvo],None,[5 baunvo],None,[6 baunvo],None,...,[14 baunvo],[15 baunvo],[16 baunvo],None,[18 baunvo],"[grundflächenzahl, grz]","[geschoßflächenzahl, geschoßfläche, gfz]",[21 baunvo],[stellplätze],None
4,24451_0.jpg,[1 baunvo],[2 baunvo],[3 baunvo],[4 baunvo],None,[5 baunvo],None,[6 baunvo],None,...,[14 baunvo],[15 baunvo],[16 baunvo],[17 baunvo],[18 baunvo],[grz],[20 baunvo],[21 baunvo],[stellplätze],None


And inspect keyword coverage across all files.

In [8]:
result_df.count()

filename      503
baunvo-1       46
baunvo-2       49
baunvo-3       42
baunvo-4       33
baunvo-4a       1
baunvo-5       36
baunvo-5a       0
baunvo-6       28
baunvo-6a       1
baunvo-7       18
baunvo-8       34
baunvo-9       24
baunvo-10      24
baunvo-11      20
baunvo-12      29
baunvo-13      13
baunvo-13a      1
baunvo-14      18
baunvo-15      19
baunvo-16      22
baunvo-17      10
baunvo-18      15
baunvo-19      60
baunvo-20      51
baunvo-21      17
baunvo-21a     36
13b             2
dtype: int64